In [1]:
using MAT
using DynamicPolynomials, LinearAlgebra, SparseArrays

data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
D = matread(data*"/WeightsDigits2.mat");
W1 = D["W1"]; W2 = D["W2"]; c = D["c"]; x_bar=D["x_bar"]; y_bar=D["y_bar"]
eps = 0.1;

m1=size(W1,2)
m2=size(W2,2)
m3=size(W2,1)

10

In [2]:
y_bar+1

9

In [71]:
@polyvar x1[1:m1] x2[1:m2] x3[1:m3]# variables


f=(c[y_bar+1,:]-c[10,:])'*x3

g=Vector{Polynomial{true,Float64}}([])
h=Vector{Polynomial{true,Float64}}([])

pol=1.0*x1[1]

for j in 1:m2
    pol=x2[j]-sum(W1[j,r]*x1[r] for r=1:m1)
    append!(g,[pol])
    append!(h,[x2[j]*pol])
end
for j in 1:m3
    pol=x3[j]-sum(W2[j,r]*x2[r] for r=1:m2)
    append!(g,[pol])
    append!(h,[x3[j]*pol])
end

for t in 1:m1
    append!(g,[-x1[t]+x_bar[1,t]+eps])
end

m=length(g)
l=length(h)

f=f([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])

for j in 1:m
    g[j]=g[j]([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])
end

for j in 1:l
    h[j]=h[j]([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])
end


x=[x1;x2;x3]; n=length(x)
println("***Problem setting***")
println("Number of variables: n=",n)
println("Number of inequality constraints: m=",m)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=84
Number of inequality constraints: m=84
Number of equality constraints: l=20


In [72]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);


k=0
s=76
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))+1

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=0
Sparsity order: s=76
Sparsity order: d=2
------------------------------------------------------
The clique sizes of varibles:
[74, 11]
[1, 10]
------------------------------------------------------
  Number of cliques: p=

11
  Largest clique size: u=74
 Maximal matrix size:75
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2970            
  Cones                  : 0               
  Scalar variables       : 9126            
  Matrix variables       : 11              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   :                 
  Objective sense        : max        

In [73]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);


k=0
s=50
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))+1

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=0
Sparsity order: s=50
Sparsity order: d=2
------------------------------------------------------
The clique sizes of varibles:
[74, 11]
[1, 10]
------------------------------------------------------
  Number of cliques: p=

11
  Largest clique size: u=74
 Maximal matrix size:50
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2970            
  Cones                  : 0               
  Scalar variables       : 9126            
  Matrix variables       : 36              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   :                 
  Objective sense        : max        

In [74]:
using SparseArrays, TSSOS

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=1

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS="MD",TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[11]
[74]
------------------------------------------------------
Obtained the variable cliques in 0.001097885 seconds. The maximal size of cliques is 11.
Assembling the SDP...
There are 954 affine constraints.
SDP assembling time: 0.004445411 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 954             
  Cones                  : 0               
  Scalar variables       : 189             
  Matrix variables       : 74              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.


6   2.7e-02  3.6e-04  1.4e-04  1.11e+00   -1.557781850e+01  -1.571156607e+01  3.6e-04  0.11  
7   6.5e-03  8.8e-05  2.7e-05  5.30e-01   -2.550741363e+01  -2.559725031e+01  8.8e-05  0.13  
8   2.1e-03  2.9e-05  9.2e-06  -2.43e-02  -4.135802399e+01  -4.146018032e+01  2.9e-05  0.14  
9   6.1e-04  8.2e-06  3.1e-06  -2.04e-01  -6.599472751e+01  -6.613341605e+01  8.2e-06  0.16  
10  2.2e-04  2.9e-06  1.4e-06  -2.81e-01  -9.987760482e+01  -1.000926084e+02  2.9e-06  0.17  
11  5.1e-05  6.9e-07  3.6e-07  -6.28e-02  -1.493975219e+02  -1.496620646e+02  6.9e-07  0.19  
12  1.5e-05  2.1e-07  1.5e-07  -3.78e-01  -2.485833813e+02  -2.491155059e+02  2.1e-07  0.21  
13  3.6e-06  4.9e-08  3.9e-08  -1.25e-02  -3.641011689e+02  -3.647254921e+02  4.9e-08  0.23  
14  8.2e-07  1.1e-08  1.1e-08  -3.28e-01  -6.156921249e+02  -6.168475948e+02  1.1e-08  0.25  
15  2.1e-07  2.3e-09  3.1e-09  -3.74e-01  -1.009327718e+03  -1.011258293e+03  2.3e-09  0.27  
16  5.2e-08  5.8e-10  1.0e-09  -3.59e-01  -1.609271141e+03  

In [28]:
binomial(11+1,1)

12

In [20]:
using SparseArrays, TSSOS

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=2

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS="MD",TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[74, 11]
[1, 10]
------------------------------------------------------
Obtained the variable cliques in 0.002665221 seconds. The maximal size of cliques is 74.


Assembling the SDP...
There are 1430065 affine constraints.
SDP assembling time: 19.855059978 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1430065         
  Cones                  : 0               
  Scalar variables       : 29281           
  Matrix variables       : 179             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.22            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.74    
MOSEK error 1051: Out o

Mosek.MosekError: Mosek.MosekError(1051, "Out of space.")

In [22]:
binomial(74+2,2)

2850